## Description
We want to find the closest airborne aeroplane to any given position in North America or Europe. To assist in this we can use an API which will give us the data on all currently airborne commercial aeroplanes in these regions.

OpenSky's Network API can return to us all the data we need in a JSON format.

https://opensky-network.org/api/states/all
From this we can find the positions of all the planes and compare them to our given position.

Use the basic Euclidean distance in your calculation.

## Input
A location in latitude and longitude, cardinal direction optional

An API call for the live data on all aeroplanes

## Output
The output should include the following details on the closest airborne aeroplane:

    Geodesic distance
    Callsign
    Lattitude and Longitude
    Geometric Altitude
    Country of origin
    ICAO24 ID
    
## Challenge Inputs
Eifel Tower:

    48.8584 N
    2.2945 E
John F. Kennedy Airport:
    
    40.6413 N
    73.7781 W
## Bonus
Replace your distance function with the geodesic distance formula, which is more accurate on the Earth's surface.

In [17]:
import requests
import pandas as pd
from math import sin,cos,radians


link = r'https://opensky-network.org/api/states/all'

cols = """icao24,callsign,origin_country,time_position,last_contact,longitude,latitude,geo_altitude,on_ground,velocity,heading,vertical_rate,sensors,baro_altitude,squawk,spi,position_source""".split(',')
keep = """icao24,callsign,origin_country,longitude,latitude,geo_altitude""".split(',')

def get_closest(longitude, latitude):
    
    r_earth = 6378136
    b = (r_earth,radians(longitude),radians(latitude))
    
    def rad(F,p):
        F['r_' + p] = F[p].apply(lambda x: radians(x))

    def euclid_polar(a):
        # polar coordinates: radius, angle1, angle2
        ar, a1, a2 = a
        br, b1, b2 = b
        return 2**0.5 * ar *(1 - sin(a2)*sin(b2)*cos(a1 - b1) - cos(a2)*cos(b2))**0.5
        return (ar**2 + br**2 - 2*ar*br*(sin(a2)*sin(b2)*cos(a1 - b1) + cos(a2)*cos(b2)))**0.5
  
    f = requests.get(link).json()
    F = pd.DataFrame(f['states'], columns = cols)
    
    rad(F,'longitude')
    rad(F,'latitude')
    F['altitude'] = F['geo_altitude'].apply(lambda x: x + r_earth)
    F['euclid_dist'] = F[['altitude','r_longitude','r_latitude']].apply(euclid_polar, axis = 1)
    F.sort_values(by=['euclid_dist'], inplace = True)

    return F[keep + ['euclid_dist']].head(3)

get_closest(2.2945, 48.8584)
# get_closest(-73.7781, 40.6413)

,icao24,callsign,origin_country,longitude,latitude,geo_altitude,euclid_dist
628,344291,VLG242N,Spain,2.2841,48.9067,3962.40,5450.395702
729,3944f5,AFR1642,France,2.3480,48.7791,4899.66,9908.041979
264,3946ec,AFR14FR,France,2.3092,48.9808,914.40,13683.181596


In [100]:
import requests
import pandas as pd
from math import sin,cos,radians

text = """Country of origin:      {2}
Callsign:               {1}
ICAO24 ID:              {0}  
Lattitude:              {3:>20.3f}
Longitude:              {4:>20.3f}
Euclidean distance (m): {6:>16.0f}
Geometric Altitude (m): {5:>16.0f}
"""
r_earth = 6378136

def euclid_polar(a1, a2, b1, b2):
    a1, a2, b1, b2 = map(radians,[a1, a2, b1, b2])
    return r_earth * (2*(1- sin(a2) * sin(b2) * cos(a1 - b1) - cos(a2) * cos(b2)))**0.5

def find_nearest_airplane(lattitude,longitude):
    link = r'https://opensky-network.org/api/states/all'
    F = pd.DataFrame(requests.get(link).json()['states'])[[0, 1, 2, 5, 6, 7]]
    
    F[8] = F[[5,6]].apply(lambda x: euclid_polar(*x,lattitude,longitude), axis = 1)
    F.sort_values(by=[8], inplace = True)
    print(text.format(*list(F.iloc[0])))
    
find_nearest_airplane(2.2945, 48.8584)
find_nearest_airplane(-73.7781, 40.6413)

Country of origin:      United Kingdom
Callsign:               EZY45PB 
ICAO24 ID:              400dae  
Lattitude:                             2.154
Longitude:                            48.899
Euclidean distance (m):            12615
Geometric Altitude (m):             3962

Country of origin:      United States
Callsign:               TEST1234
ICAO24 ID:              adfbde  
Lattitude:                           -73.759
Longitude:                            40.635
Euclidean distance (m):             1559
Geometric Altitude (m):             3475



In [20]:
pi = 3.1564
f'{pi:.2f}'

'3.16'

In [4]:
a1, a2, b1, b2 = 0,1,2,3

a1, a2, b1, b2 = map(radians,[a1, a2, b1, b2])

a1, a2, b1, b2

(0.0, 0.017453292519943295, 0.03490658503988659, 0.05235987755982989)

In [93]:
import urllib.request, json
from math import acos, sin, cos, radians
with urllib.request.urlopen("https://opensky-network.org/api/states/all") as url:
    data = json.loads(url.read().decode())['states']

EARTH_RAD = 6371
def geo_dist(x, y):
    xrad = [radians(d) for d in x]
    yrad = [radians(d) for d in y]
    return EARTH_RAD*acos(sin(xrad[0])*sin(yrad[0])+
                          cos(xrad[0])*cos(yrad[0])*
                          cos(abs(xrad[1]-yrad[1])))

def get_closest(longitude,lattitude):
    closest = data[0],10000000
    for plane in data:
        if None in plane[5:7]: continue
        dist = geo_dist(plane[5:7],[longitude,lattitude])
        if dist<closest[1]:
            closest = plane, dist
    print("closest plane to ",longitude,"E ",lattitude,"N")
    print("Geodesic distance:", closest[1])
    print("Callsign:", closest[0][1])
    print("Longitude and Lattitude:", closest[0][5],"E", closest[0][6],"N")
    print("Geometric Altitude:", closest[0][7])
    print("Country of origin:", closest[0][2])
    print("ICAO24 ID:", closest[0][0])

print("EIFEL TOWER\n------------")
get_closest(2.2945,48.8584)


EIFEL TOWER
------------
closest plane to  2.2945 E  48.8584 N
Geodesic distance: 13.854824364521017
Callsign: AFR770E 
Longitude and Lattitude: 2.3234 E 48.9797 N
Geometric Altitude: 1021.08
Country of origin: France
ICAO24 ID: 393d84


In [85]:
from scipy.spatial import distance


EIFEL = (48.8584, 2.2945)
JFK = (40.6413, 73.7781)

states = requests.get(link).json()
dists_eifel, dists_jfk = [], []
for s in states['states']:
    co, cs, ic, ga = s.origin_country, s.callsign, s.icao24, s.geo_altitude
    latitude, longitude = s.latitude, s.longitude
    if latitude and longitude:
        eifel_d = distance.euclidean(EIFEL, (latitude, longitude))
        jfk_d = distance.euclidean(JFK, (latitude, longitude))
        dists_eifel.append((eifel_d, cs, latitude, longitude, ga, co, ic))
        dists_jfk.append((jfk_d, cs, latitude, longitude, ga, co, ic))

print("Closest to the Eifel Tower:", min(dists_eifel, key=lambda f: f[0]))
print("Closest to JFK:", min(dists_jfk, key=lambda f: f[0]))

AttributeError: 'list' object has no attribute 'origin_country'

In [3]:
class Node:
    def __init__(self, label=None, data=None):
        self.label = label
        self.data = data
        self.children = dict()
    
    def addChild(self, key, data=None):
        if not isinstance(key, Node):
            self.children[key] = Node(key, data)
        else:
            self.children[key.label] = key
    
    def __getitem__(self, key):
        return self.children[key]

class Trie:
    def __init__(self):
        self.head = Node()
    
    def __getitem__(self, key):
        return self.head.children[key]
    
    def add(self, word):
        current_node = self.head
        word_finished = True
        
        for i in range(len(word)):
            if word[i] in current_node.children:
                current_node = current_node.children[word[i]]
            else:
                word_finished = False
                break
        
        # For ever new letter, create a new child node
        if not word_finished:
            while i < len(word):
                current_node.addChild(word[i])
                current_node = current_node.children[word[i]]
                i += 1
        
        # Let's store the full word at the end node so we don't need to
        # travel back up the tree to reconstruct the word
        current_node.data = word
    
    def has_word(self, word):
        if word == '':
            return False
        if word == None:
            raise ValueError('Trie.has_word requires a not-Null string')
        
        # Start at the top
        current_node = self.head
        exists = True
        for letter in word:
            if letter in current_node.children:
                current_node = current_node.children[letter]
            else:
                exists = False
                break
        
        # Still need to check if we just reached a word like 't'
        # that isn't actually a full word in our dictionary
        if exists:
            if current_node.data == None:
                exists = False
        
        return exists
    
    def start_with_prefix(self, prefix):
        """ Returns a list of all words in tree that start with prefix """
        words = list()
        if prefix == None:
            raise ValueError('Requires not-Null prefix')
        
        # Determine end-of-prefix node
        top_node = self.head
        for letter in prefix:
            if letter in top_node.children:
                top_node = top_node.children[letter]
            else:
                # Prefix not in tree, go no further
                return words
        
        # Get words under prefix
        if top_node == self.head:
            queue = [node for key, node in top_node.children.iteritems()]
        else:
            queue = [top_node]
        
        # Perform a breadth first search under the prefix
        # A cool effect of using BFS as opposed to DFS is that BFS will return
        # a list of words ordered by increasing length
        while queue:
            current_node = queue.pop()
            if current_node.data != None:
                # Isn't it nice to not have to go back up the tree?
                words.append(current_node.data)
            
            queue = [node for key,node in current_node.children.items()] + queue
        
        return words
    
    def getData(self, word):
        """ This returns the 'data' of the node identified by the given word """
        if not self.has_word(word):
            raise ValueError('{} not found in trie'.format(word))
        
        # Race to the bottom, get data
        current_node = self.head
        for letter in word:
            current_node = current_node[letter]
        
        return current_node.data

if __name__ == '__main__':
    """ Example use """
    trie = Trie()
    words = 'hello goodbye help gerald gold tea ted team to too tom stan standard money'
    for word in words.split():
        trie.add(word)
    print( "'goodbye' in trie: ", )
    print(trie.start_with_prefix('g'))
    print(trie.start_with_prefix('to'))

'goodbye' in trie:  True
['gold', 'gerald', 'goodbye']
['to', 'tom', 'too']


In [5]:
%timeit trie.has_word('goodbye')

The slowest run took 4.70 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 3.27 µs per loop


In [7]:
s = set('hello goodbye help gerald gold tea ted team to too tom stan standard money'.split(' ')) 
%timeit 'goodbye' in s

The slowest run took 19.53 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 143 ns per loop
